In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as ssp
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics, model_selection
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
import lightgbm
from lightgbm.sklearn import LGBMClassifier
import os 
import re
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier 

C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# I will use cross_val_score on XGBoost to select 100,200 or 300 for each preprocess => actually selected so ignore that part

In [2]:
work_train_w2v = {} 
work_test_w2v = {}
pre_process=["w2v_100.csv","w2v_200.csv","w2v_300.csv"]
path="checkpoints_databases/"
for f in pre_process:
    work_train_w2v[re.sub("\.csv","",f)] = pd.read_csv(path+"new_stem_working_train_"+f)
    work_test_w2v[re.sub("\.csv","",f)] = pd.read_csv(path+"new_stem_working_test_"+f)

In [3]:
work_train_tfidf = {} 
work_test_tfidf = {}
pre_process=["tfidf_tsvd_100.csv","tfidf_tsvd_200.csv","tfidf_tsvd_300.csv"]
path="checkpoints_databases/"
for f in pre_process:
    work_train_tfidf[re.sub("\.csv","",f)] = pd.read_csv(path+"new_stem_working_train_"+f)
    work_test_tfidf[re.sub("\.csv","",f)] = pd.read_csv(path+"new_stem_working_test_"+f)

In [ ]:
#we keep the lemmatization bio w2v
work_train_bio = {} 
work_test_bio = {}
pre_process=["bio.csv"]
path="checkpoints_databases/"
for f in pre_process:
    work_train_bio[re.sub("\.csv","",f)] = pd.read_csv(path+"new_working_train_"+f)
    work_test_bio[re.sub("\.csv","",f)] = pd.read_csv(path+"new_working_test_"+f)

In [8]:
clf_xgb=XGBClassifier(max_depth=5, objective="multi:softprob",seed=26)
for name in work_train_w2v:
    h=cross_val_score(clf_xgb,np.array(work_train_w2v[name]),y,cv=kf,n_jobs=-1,scoring="neg_log_loss")
    print("mean"+name+" "+str(h.mean()),
         "std:"+name+" "+str(h.std()))
#

meanw2v_100 -1.08591915345 std:w2v_100 0.0541873570702
meanw2v_200 -1.07376934966 std:w2v_200 0.0372816273367
meanw2v_300 -1.06437402745 std:w2v_300 0.0367764336005


In [9]:
for name in work_train_tfidf:
    h=cross_val_score(clf_xgb,np.array(work_train_tfidf[name]),y,cv=kf,n_jobs=-1,scoring="neg_log_loss")
    print("mean"+name+" "+str(h.mean()),
         "std:"+name+" "+str(h.std()))
#

meantfidf_tsvd_100 -1.0169704023 std:tfidf_tsvd_100 0.0365495778442
meantfidf_tsvd_200 -1.02394548835 std:tfidf_tsvd_200 0.0384426689575
meantfidf_tsvd_300 -1.02594316006 std:tfidf_tsvd_300 0.0390292436386


# things start here

In [2]:
new_train=pd.read_csv("checkpoints_databases/new_working_train.csv")
new_test=pd.read_csv("checkpoints_databases/new_working_test.csv")

In [3]:
y=np.array(new_train.Class)-1

In [4]:
ID_train=new_train.ID
ID_test=new_test.ID

In [5]:
kf = model_selection.StratifiedKFold(n_splits=5, random_state=26, shuffle=True)

# GRID SEARCH PHASE ALL ALGOS

In [7]:
work_train= {} 
work_test = {}
#To complete
path="checkpoints_databases/"
work_train["bio"] = pd.read_csv(path+"new_working_train_bio.csv")
work_test["bio"] = pd.read_csv(path+"new_working_test_bio.csv")
work_train["full_tfidf_100"] = pd.read_csv(path+"new_stem_working_train_full_tfidf_100.csv")
work_test["full_tfidf_100"] = pd.read_csv(path+"new_stem_working_test_full_tfidf_100.csv")

work_train["wind_tfidf_100"] = pd.read_csv(path+"new_stem_working_train_wind_tfidf_100.csv")
work_test["wind_tfidf_100"] = pd.read_csv(path+"new_stem_working_test_wind_tfidf_100.csv")

work_train["w2v_300"] = pd.read_csv(path+"new_stem_working_train_w2v_300.csv")
work_test["w2v_300"] = pd.read_csv(path+"new_stem_working_test_w2v_300.csv")


In [8]:
feat_train=pd.read_csv("../window/w_meta_features/meta_train_l1l2.csv")
feat_test=pd.read_csv("../window/w_meta_features/meta_test_l1l2.csv")
feat_train=feat_train.drop("ID",axis=1)
feat_test=feat_test.drop("ID",axis=1)
feat_ext_train=pd.read_csv("w_meta_features/new_working_train_ext.csv")
feat_ext_test=pd.read_csv("w_meta_features/new_working_test_ext.csv")

In [9]:
#GO features + text embeddings
work_train_final={}
work_test_final={}
for name in work_train:
    work_train_final[name]=pd.concat((work_train[name],feat_ext_train),axis=1)
    work_test_final[name]=pd.concat((work_test[name],feat_ext_test),axis=1)

In [10]:
for name in work_train_final:
    print(name) #just to check that the ordering is not doing shit

bio
full_tfidf_100
wind_tfidf_100
w2v_300


# the gridsearch for w2vbio lemmatized is already done in the other modelling script

In [ ]:
clf_xgb=XGBClassifier(max_depth=3, objective="multi:softprob",seed=26)
param_test= {
    "max_depth" : [3,5,7],
    "min_child_weight" : [1,3,5],
    "subsample":[0.8,0.9,1],
    "colsample_bytree":[0.8,0.9,1]
}
for name in work_train_final:
    gsearch=GridSearchCV(estimator=clf_xgb, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf,verbose=True)
    gsearch.fit(np.array(work_train_final[name]),y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)
    
#bio:
#f_tfidf:
#w_tfidf:
#w2v:
    
#bio : {'colsample_bytree': 0.8, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 100, 'subsample': 0.9} : 0.9385
#tfidf: 
#w2v: 

Fitting 5 folds for each of 162 candidates, totalling 810 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 107.7min


In [ ]:
clf_lgbm=LGBMClassifier(seed=26)
param_test= {
    'n_estimators': [8,24,36,48],
    'num_leaves': [6,12,16,22],
    'boosting_type' : ['gbdt'],
    'colsample_bytree' : [0.7,0.8,0.9],
    'subsample' : [0.7,0.8,0.9]
    }
for name in work_train_final:
    gsearch=GridSearchCV(estimator=clf_lgbm, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf,verbose=True)
    gsearch.fit(np.array(work_train_final[name]),y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)
#bio: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 48, 'num_leaves': 16, 'subsample': 0.9}-0.940016998893
#f_tfidf:{'boosting_type': 'gbdt', 'colsample_bytree': 0.9, 'n_estimators': 48, 'num_leaves': 16, 'subsample': 0.9}-0.923197719885
#w_tfidf:{'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 48, 'num_leaves': 22, 'subsample': 0.9}-0.938995537783
#w2v:
    
#bio :{'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 48, 'num_leaves': 22, 'subsample': 0.9}
#-0.950704488318
#tfidf : {'boosting_type': 'gbdt', 'colsample_bytree': 0.9, 'n_estimators': 48, 'num_leaves': 22, 'subsample': 0.8}
#-0.93280968028
#w2v : {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 48, 'num_leaves': 22, 'subsample': 0.9}
#-0.948655365308

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   20.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 10.6min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed: 19.8min finished


bio
[mean: -1.55490, std: 0.01438, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leaves': 6, 'subsample': 0.7}, mean: -1.55617, std: 0.01565, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leaves': 6, 'subsample': 0.8}, mean: -1.55400, std: 0.01066, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leaves': 6, 'subsample': 0.9}, mean: -1.48765, std: 0.01384, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leaves': 12, 'subsample': 0.7}, mean: -1.48021, std: 0.01472, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leaves': 12, 'subsample': 0.8}, mean: -1.47836, std: 0.01221, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leaves': 12, 'subsample': 0.9}, mean: -1.46263, std: 0.01420, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leaves': 16, 's

C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   14.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed: 11.7min finished


full_tfidf_100
[mean: -1.55173, std: 0.01291, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leaves': 6, 'subsample': 0.7}, mean: -1.54892, std: 0.01458, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leaves': 6, 'subsample': 0.8}, mean: -1.54990, std: 0.01558, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leaves': 6, 'subsample': 0.9}, mean: -1.47456, std: 0.01212, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leaves': 12, 'subsample': 0.7}, mean: -1.46893, std: 0.01672, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leaves': 12, 'subsample': 0.8}, mean: -1.46555, std: 0.01470, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leaves': 12, 'subsample': 0.9}, mean: -1.45017, std: 0.01305, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leav

C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed: 11.8min finished


wind_tfidf_100
[mean: -1.57294, std: 0.01220, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leaves': 6, 'subsample': 0.7}, mean: -1.57451, std: 0.01339, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leaves': 6, 'subsample': 0.8}, mean: -1.57029, std: 0.01107, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leaves': 6, 'subsample': 0.9}, mean: -1.50799, std: 0.01627, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leaves': 12, 'subsample': 0.7}, mean: -1.49878, std: 0.01334, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leaves': 12, 'subsample': 0.8}, mean: -1.49331, std: 0.01404, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leaves': 12, 'subsample': 0.9}, mean: -1.48350, std: 0.01210, params: {'boosting_type': 'gbdt', 'colsample_bytree': 0.7, 'n_estimators': 8, 'num_leav

C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   28.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 16.4min


In [ ]:
clf_log=LogisticRegression()
param_test= {
    "C" : [0.1,1,3,5,10,15,20],
    "penalty" : ["l1","l2"]
}
for name in work_train_final:
    gsearch=GridSearchCV(estimator=clf_log, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf,verbose=True)
    gsearch.fit(np.array(work_train_final[name]),y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)
    
#bio:
#f_tfidf:
#w_tfidf:
#w2v:
    
#bio : {'C': 10, 'penalty': 'l2'}
#tfidf : {'C': 9, 'penalty': 'l2'}
#w2v : {'C': 5, 'penalty': 'l2'}

In [ ]:
clf_dt=RandomForestClassifier()
param_test={
    "max_depth":[11,13,15,17,19],
    "n_estimators":[250,300,350]
}
for name in work_train_final:
    gsearch=GridSearchCV(estimator=clf_dt, param_grid = param_test,scoring="neg_log_loss",n_jobs=-1,iid=False, cv=kf,verbose=True)
    gsearch.fit(np.array(work_train_final[name]),y)
    print(name)
    print(gsearch.grid_scores_)
    print(gsearch.best_params_)
    print(gsearch.best_score_)
    
#bio:
#f_tfidf:
#w_tfidf:
#w2v:

#bio : {'max_depth': 15, 'n_estimators': 300} -0.957419688259 ok
#tfidf : {'max_depth': 13, 'n_estimators': 350} -0.920825809843 ok 
#w2v :{'max_depth': 15, 'n_estimators': 300} -0.972048498265

# TRAINING PHASE ALL ALGOS 

In [23]:
def model_gen(X,X_test,y,classifier,file,five_fold_predict=True):
    #if not os.path.exists("scores/"+file):
    #   os.makedirs("scores/"+file)
    if five_fold_predict:
        fold = 0
        y_test=0
        for train_index, test_index in kf.split(X, y):
        
            fold += 1

            X_train, X_valid    = X[train_index],   X[test_index]
            y_train, y_valid    = y[train_index],   y[test_index]

            print("Fold", fold, X_train.shape, X_valid.shape)

            clf=classifier
            clf.fit(X_train,y_train)
            p_test = clf.predict_proba(X_test)
            y_test += p_test/5

    classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
    subm = pd.DataFrame(y_test, columns=classes)
    subm['ID'] = ID_test
    
    subm.to_csv("new_scores/new_stack_test/new_stem_{}.csv".format(file),index=False)
    
    print("cross_val sur train ") #peut etre que to array est exclusivement pour les xgb

    y_pred=cross_val_predict(estimator=clf,X=X,y=y,cv=kf,method="predict_proba")
    subm1 = pd.DataFrame(y_pred, columns=classes)
    subm1['ID'] = ID_train
    subm1.to_csv("new_scores/new_stack_train/new_stem_{}.csv".format(file),index=False)


In [138]:
dic_xgb={"xgb_bio":XGBClassifier(colsample_bytree=0.8,max_depth=7,min_child_weight=3,n_estimators=100,subsample=0.8,objective="multi:softprob",seed=26),
        "xgb_w_tfidf":XGBClassifier(colsample_bytree=0.8,max_depth=5,min_child_weight=3,n_estimators=100,subsample=1,objective="multi:softprob",seed=26),
        "xgb_f_tfidf"::XGBClassifier(colsample_bytree=0.8,max_depth=5,min_child_weight=3,n_estimators=100,subsample=1,objective="multi:softprob",seed=26),
        "xgb_w2v":XGBClassifier(colsample_bytree=0.8,max_depth=5,min_child_weight=3,n_estimators=100,subsample=0.8,objective="multi:softprob",seed=26)}
dic_lgbm={"lgbm_bio":LGBMClassifier(boosting_type="gbdt",colsample_bytree=0.8,n_estimators=48,num_leaves=22,subsample=0.8,seed=26),
        "lgbm_w_tfidf":LGBMClassifier(boosting_type="gbdt",colsample_bytree=0.8,n_estimators=48,num_leaves=22,subsample=0.8,seed=26),
        "lgbm_f_tfidf":LGBMClassifier(boosting_type="gbdt",colsample_bytree=0.8,n_estimators=48,num_leaves=22,subsample=0.8,seed=26),
        "lgbm_w2v":LGBMClassifier(boosting_type="gbdt",colsample_bytree=0.8,n_estimators=48,num_leaves=22,subsample=0.8,seed=26)}
dic_lr={"lr_bio":LogisticRegression(C=10,penalty="l1"),
        "lr_w_tfidf":LogisticRegression(C=10,penalty="l1"),
        "lr_f_tfidf":LogisticRegression(C=10,penalty="l1"),
        "lr_w2v":LogisticRegression(C=1,penalty="l1")}
dic_rf={"rf_bio":RandomForestClassifier(n_estimators=400,max_depth=25,random_state=26),
        "rf_w_tfidf":RandomForestClassifier(n_estimators=400,max_depth=20,random_state=26),
        "rf_f_tfidf":RandomForestClassifier(n_estimators=400,max_depth=20,random_state=26),
        "rf_w2v":RandomForestClassifier(n_estimators=300,max_depth=15,random_state=26)}

print("xgboost here")
for clf,name in zip(dic_xgb.keys(),work_train_final.keys()):
    model_gen(X=np.array(work_train_final[name]),X_test=np.array(work_test_final[name]),y=y,classifier=dic_xgb[clf],file=clf)
print("lgbm here")
for clf,name in zip(dic_lgbm.keys(),work_train_final.keys()):
    model_gen(X=np.array(work_train_final[name]),X_test=np.array(work_test_final[name]),y=y,classifier=dic_lgbm[clf],file=clf)
print("logreg here")
for clf,name in zip(dic_lr.keys(),work_train_final.keys()):
    model_gen(X=np.array(work_train_final[name]),X_test=np.array(work_test_final[name]),y=y,classifier=dic_lr[clf],file=clf)
print("random forest here")
for clf,name in zip(dic_rf.keys(),work_train_final.keys()):
    model_gen(X=np.array(work_train_final[name]),X_test=np.array(work_test_final[name]),y=y,classifier=dic_rf[clf],file=clf)



xgboost here
Fold 1 (2946, 142) (743, 142)
Fold 2 (2949, 142) (740, 142)
Fold 3 (2952, 142) (737, 142)
Fold 4 (2954, 142) (735, 142)
Fold 5 (2955, 142) (734, 142)
cross_val sur train 
Fold 1 (2946, 142) (743, 142)
Fold 2 (2949, 142) (740, 142)
Fold 3 (2952, 142) (737, 142)
Fold 4 (2954, 142) (735, 142)
Fold 5 (2955, 142) (734, 142)
cross_val sur train 
Fold 1 (2946, 243) (743, 243)
Fold 2 (2949, 243) (740, 243)
Fold 3 (2952, 243) (737, 243)
Fold 4 (2954, 243) (735, 243)
Fold 5 (2955, 243) (734, 243)
cross_val sur train 
lgbm here
Fold 1 (2946, 142) (743, 142)
Fold 2 (2949, 142) (740, 142)
Fold 3 (2952, 142) (737, 142)
Fold 4 (2954, 142) (735, 142)
Fold 5 (2955, 142) (734, 142)
cross_val sur train 
Fold 1 (2946, 142) (743, 142)
Fold 2 (2949, 142) (740, 142)
Fold 3 (2952, 142) (737, 142)
Fold 4 (2954, 142) (735, 142)
Fold 5 (2955, 142) (734, 142)
cross_val sur train 
Fold 1 (2946, 243) (743, 243)
Fold 2 (2949, 243) (740, 243)
Fold 3 (2952, 243) (737, 243)
Fold 4 (2954, 243) (735, 243)
Fo

In [ ]:
work_train_final